# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
prices_bought = [5, 15, 10, 40]

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    flags = np.isin(recommended_list, bought_list)

    recall = np.dot(flags, prices_recommended) / np.sum(prices_bought)
    
    return recall

In [4]:
recall_at_k(recommended_list,bought_list,k = 5)

0.5

In [5]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.7142857142857143

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [6]:
bought = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [7]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
  
    recommended_list = recommended_list[:k]
    ranks = 0.
    for item_rec in recommended_list:
        if item_rec not in bought_list:
            ranks += 1
            continue
        else:
            ranks += 1
            return 1 / ranks

In [8]:
reciprocal_rank_at_k(recommended, bought, 8)

0.5

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [9]:
def ndcg_at_k(rec_list, bought_list, k=5):
    bought_list = np.array(bought_list[:k])
    recommended_list = np.array(rec_list[:k])
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    ideal_den = np.array([i if i < 3 else np.log2(i) for i in range(1, k + 1)])
    ideal_ones = np.ones_like(ideal_den)
    
    n_ideal_ones = np.zeros_like(ideal_den)
    n_ideal_ones[relevant_indexes] = 1
    dcg = np.sum(n_ideal_ones / ideal_den) / k
    
    ideal_dcg = np.sum(ideal_ones / ideal_den) / k
    return dcg / ideal_dcg
        

In [12]:
ndcg_at_k(recommended_list, bought_list,5)

0.489938890671454

In [13]:
ndcg_at_k(recommended, bought,5)

0.36939097926142717